In [ ]:
import numpy as np
np.random.seed(22)
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from model import VideoObjectSegmentationModel
from dataset import Dataset

In [ ]:
# VOS-v4
models_path = {
    "pong#1"    : "wandb/run-20220707_191800-124hx1ez/files/PongNoFrameskip-v4.pt",
    "pongf#1"   : "wandb/run-20220707_191800-124hx1ez/files/PongNoFrameskip-v4_final.pt",
    
    "pong#2"    : "wandb/run-20220707_191809-s28zyjxj/files/PongNoFrameskip-v4.pt",
    "pongf#2"   : "wandb/run-20220707_191809-s28zyjxj/files/PongNoFrameskip-v4_final.pt",
    
    "pong#3"    : "wandb/run-20220707_192109-3jov8jat/files/PongNoFrameskip-v4.pt",
    "pongf#3"   : "wandb/run-20220707_192109-3jov8jat/files/PongNoFrameskip-v4_final.pt",
}

In [ ]:
env = "PongNoFrameskip-v4" # "MsPacmanNoFrameskip-v4"
batch_size = 64
H = W = 84
num_frames = 2
model_path = models_path["pong#1"]
model = VideoObjectSegmentationModel("cpu")
model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.eval();

In [ ]:
dataset = Dataset(env, 16, num_frames, H, W)
inp = dataset.get_batch("train")
x0_ = model(inp)
x0 = torch.unsqueeze(inp[:, 0, :, :], 1)
x1 = torch.unsqueeze(inp[:, 1, :, :], 1)
loss = model.compute_loss(x0, x0_)
# print(loss)
# print(x0)
# print(x0_)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 4))
idx = 14
ax[0].imshow(x0[idx].permute([1, 2, 0]).detach().numpy(), cmap='gray')
ax[1].imshow(x1[idx].permute([1, 2, 0]).detach().numpy(), cmap='gray')
ax[2].imshow(x0_[idx].permute([1, 2, 0]).detach().numpy(), cmap='gray')

In [ ]:
num_f = 5
masks = model.object_masks
fig, ax = plt.subplots(1, num_f, figsize=(25, 15))
for i in range(num_f):
    ax[i].imshow(masks[idx, i, :].detach().numpy(), cmap='gray')

In [ ]:
x = torch.zeros(84, 84)
for i in range(model.K):
    x = x + model.object_masks[idx, i, :].detach().numpy()
plt.imshow(x, cmap='gray')